In [1]:
import pandas as pd
import numpy as np

In [2]:
def load_and_clean():
    # Load files

    data_dir = r"C:\Users\rajas\Documents\ADS\SII\Big_Data_Bowl"
    routes_path = f"{data_dir}\\wr_routes_embeddings.csv"
    metrics_path = f"{data_dir}\\air_play_iq_metrics_weighted.csv"
    preds_path = f"{data_dir}\\postthrow_predictions_Completed_Pass.csv"
    df_metrics = pd.read_csv(metrics_path)
    df_preds = pd.read_csv(preds_path)
    df_routes = pd.read_csv(routes_path)

    # Cleaning Function
    def clean_wr(df):
        # Filter for WRs only
        if 'player_position' in df.columns:
            df = df[df['player_position'] == 'WR'].copy()
        
        # Remove Junk Routes (Screens/Flats/Blocking)
        if 'route_label' in df.columns:
            junk = ['SCREEN', 'FLAT', 'ANGLE', 'WHEEL', 'SHIELD']
            df = df[~df['route_label'].isin(junk)].copy()
            
        # Remove Line of Scrimmage plays
        if 'air_yards' in df.columns:
            df = df[df['air_yards'] > 0].copy()
        return df

    return clean_wr(df_metrics), df_preds, clean_wr(df_routes)

df_metrics, df_preds, df_routes = load_and_clean()

In [3]:
# INSIGHT 1: PLAYER ARCHETYPES (Style Matrix)
# Group by player
style_stats = df_metrics.groupby(['player_name']).agg({
    'RouteExecIQ': 'mean',
    'ConvergenceIQ_WR': 'mean',
    'AirPlayIQ_WR': 'mean',
    'game_id': 'count'
}).rename(columns={'game_id': 'routes_run'})

# Filter for Starters (min 40% of max volume)
min_vol = style_stats['routes_run'].max() * 0.40
df_style = style_stats[style_stats['routes_run'] > min_vol].copy()

# Calculate Percentiles to determine style
df_style['Exec_Pct'] = df_style['RouteExecIQ'].rank(pct=True)
df_style['Conv_Pct'] = df_style['ConvergenceIQ_WR'].rank(pct=True)

def define_archetype(row):
    if row['Exec_Pct'] > 0.70 and row['Conv_Pct'] > 0.70: return "Elite All-Around"
    if row['Exec_Pct'] < 0.40 and row['Conv_Pct'] > 0.70: return "Freestyle / Physical Freak"
    if row['Exec_Pct'] > 0.70 and row['Conv_Pct'] < 0.40: return "Separator / Technician"
    if row['Exec_Pct'] < 0.40 and row['Conv_Pct'] < 0.40: return "Struggling"
    return "Balanced"

df_style['Archetype'] = df_style.apply(define_archetype, axis=1)

In [4]:
# INSIGHT 2: KINGS OF THE ROUTE TREE
route_stats = df_metrics.groupby(['route_label', 'player_name']).agg({
    'AirPlayIQ_WR': 'mean',
    'game_id': 'count'
}).reset_index()

best_routes = []
for route in route_stats['route_label'].unique():
    # Get players who ran this route enough times (20% of max volume for that route)
    subset = route_stats[route_stats['route_label'] == route]
    min_route_vol = max(5, subset['game_id'].max() * 0.20)
    qualified = subset[subset['game_id'] >= min_route_vol]
    
    if not qualified.empty:
        top_player = qualified.sort_values('AirPlayIQ_WR', ascending=False).iloc[0]
        best_routes.append({
            'Route': route,
            'Best_Player': top_player['player_name'],
            'Score': round(top_player['AirPlayIQ_WR'], 3)
        })

df_specialists = pd.DataFrame(best_routes).sort_values('Route')

In [5]:
# INSIGHT 3: MODEL BEATERS (Residual Analysis)
# Merge predictions with player names
df_res = df_preds.merge(df_metrics[['game_id', 'play_id', 'player_name']], 
                        on=['game_id', 'play_id'], how='inner')

# Calculate average residual (Actual Speed - Predicted Speed)
df_beaters = df_res.groupby('player_name')['residual'].mean().reset_index()
df_beaters = df_beaters.merge(df_style[['routes_run']], on='player_name', how='inner') # Filter for starters
df_beaters = df_beaters.sort_values('residual', ascending=False).head(10)

In [6]:
print("\n--- 1. ARCHETYPE EXAMPLES ---")
print(df_style.groupby('Archetype').head(2)[['AirPlayIQ_WR', 'Archetype']])

print("\n--- 2. ROUTE SPECIALISTS ---")
print(df_specialists)

print("\n--- 3. TOP 5 MODEL BEATERS (Hidden Speed) ---")
print(df_beaters.head(5))


--- 1. ARCHETYPE EXAMPLES ---
                   AirPlayIQ_WR                   Archetype
player_name                                                
A.J. Brown             0.294057                    Balanced
Adam Thielen           0.017402                  Struggling
Alec Pierce            0.347809  Freestyle / Physical Freak
Amari Cooper           0.073184      Separator / Technician
Amon-Ra St. Brown      0.130233                    Balanced
Brandin Cooks          0.252355            Elite All-Around
Chris Godwin Jr.       0.027505      Separator / Technician
Chris Olave            0.236027  Freestyle / Physical Freak
Christian Kirk        -0.138510                  Struggling
DeVonta Smith          0.171098            Elite All-Around

--- 2. ROUTE SPECIALISTS ---
    Route            Best_Player  Score
0  CORNER          Brandin Cooks  0.655
1   CROSS      Wan'Dale Robinson  0.965
2      GO            Tyler Scott  1.203
3   HITCH          Josh Reynolds  0.623
4      IN  Donovan 